In [2]:
## Entrenamiento de modelo MLPClassifier; Escalado de características; División de datos en conjuntos de entrenamiento y validación; Evaluación del modelo con métricas de precisión y reporte; Generación de reporte de clasificación; Codificación de etiquetas de clase para target
## ----------------------------------------------------------------
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neural_network import MLPClassifier
import joblib
from sklearn.preprocessing import LabelEncoder, StandardScaler

## Sesgo (Skewness) y Curtosis (Kurtosis)

### Sesgo (Skewness)

La **sesgo** o *skewness* es una medida de la asimetría de la distribución de una variable aleatoria en torno a su media. Una distribución simétrica (por ejemplo, la normal) tiene skewness igual a cero; si la cola de la distribución se extiende más hacia la derecha (valores grandes), la skewness es positiva; si se extiende más hacia la izquierda, es negativa.

#### Definición poblacional

Para una variable aleatoria $X$ con esperanza $\mu = \mathbb{E}[X]$ y desviación estándar $\sigma = \sqrt{\mathbb{E}[(X-\mu)^2]}$, la skewness poblacional se define como el momento estandarizado de orden 3:

$$
\gamma_1 \;=\; \frac{\mathbb{E}\bigl[(X - \mu)^3\bigr]}{\sigma^3}.
$$

- $$\gamma_1 = 0$$: distribución perfectamente simétrica.  
- $$\gamma_1 > 0$$: cola derecha más larga o sesgo hacia la derecha.  
- $$\gamma_1 < 0$$: cola izquierda más larga o sesgo hacia la izquierda.  

#### Estimador muestral

Dado un conjunto de datos (muestra) $\{x_i\}_{i=1}^n$, con media muestral

$$
\bar{x} = \frac{1}{n}\sum_{i=1}^n x_i
$$

y desviación estándar muestral

$$
\hat{\sigma} = \sqrt{\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^2},
$$

un estimador común de skewness (no corregido por sesgo) es:

$$
g_1 \;=\; \frac{\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^3}{\left(\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^2\right)^{3/2}}
\;=\; \frac{\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^3}{\hat{\sigma}^3}.
$$

> Nota: este estimador puede estar sesgado para muestras pequeñas. Existen correcciones (por ejemplo la definición de Fisher–Pearson), pero la forma básica anterior es la más usada para cálculo rápido y comprensión de la asimetría de los datos.

#### ¿Para qué sirve la skewness?

- Detectar asimetría en la distribución de datos: si el sesgo es significativo, puede influir en métodos que asumen normalidad o simetría (por ejemplo, en ciertas pruebas estadísticas o en estimaciones que dependen de la asimetría).  
- Informar transformaciones de datos: al observar skewness alta (positiva o negativa), se puede considerar aplicar transformaciones (por ejemplo, log, raíz, Box–Cox) para aproximar una distribución más simétrica.  
- Análisis exploratorio de datos (EDA): como parte de la descripción de la forma de la distribución, junto con media, mediana, varianza, curtosis, etc.

---

### Curtosis (Kurtosis)

La **curtosis** o *kurtosis* mide la “forma de la cola” o el grado de concentración de la distribución en torno a la media (picudez y colas). Existen dos modos de definirla: la curtosis absoluta (o poblacional) y la **curtosis excesiva** (excess kurtosis), que compara con la distribución normal.

#### Definición poblacional de curtosis

Para la misma variable $$X$$ con media $$\mu$$ y desviación estándar $$\sigma$$, la curtosis poblacional se define como el momento estandarizado de orden 4:

$$
\gamma_2 \;=\; \frac{\mathbb{E}\bigl[(X - \mu)^4\bigr]}{\sigma^4}.
$$

Muchos textos definen la **curtosis excesiva** restando 3 (el valor de la normal), de modo que:

$$
\text{excess kurtosis} \;=\; \gamma_2 - 3.
$$

- $$\gamma_2 - 3 = 0$$: misma “picudez” y colas que la normal (mesocúrtica).  
- $$\gamma_2 - 3 > 0$$: colas más pesadas y pico más pronunciado (leptocúrtica).  
- $$\gamma_2 - 3 < 0$$: colas más ligeras y pico más achatado (platicúrtica).  

#### Estimador muestral

Para una muestra $\{x_i\}_{i=1}^n$ con media $\bar{x}$ y desviación estándar $\hat{\sigma}$, un estimador sencillo de curtosis (no ajustado) es:

$$
g_2 \;=\; \frac{\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^4}{\left(\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^2\right)^{2}}
\;=\; \frac{\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^4}{\hat{\sigma}^4}.
$$

La **curtosis excesiva muestral** se suele obtener restando 3:

$$
g_2 - 3.
$$

> Nota: existen fórmulas con corrección de sesgo para muestras pequeñas (por ejemplo, la fórmula de Fisher–Pearson ajustada con factores en función de $n$), pero la expresión anterior es la más directa para entender la “forma de la cola”.

#### ¿Para qué sirve la curtosis?

- Evaluar colas de la distribución: colas pesadas ($$\gamma_2 - 3 > 0$$) indican mayor probabilidad de valores extremos, relevante en finanzas (riesgo de pérdidas grandes), calidad de procesos (valores atípicos), etc.  
- Contrastar con normalidad: si un análisis asume normalidad, una curtosis muy distinta de 3 (excess kurtosis no nula) sugiere desviaciones que pueden afectar pruebas y modelos.  
- Complemento al análisis de dispersión: junto con varianza y skewness, da una visión más completa de la forma de los datos.

---

### Resumen de fórmulas

- **Skewness poblacional**:  
  $$
  \gamma_1 = \frac{\mathbb{E}\bigl[(X - \mu)^3\bigr]}{\sigma^3}.
  $$
- **Skewness muestral (no corregida)**:  
  $$
  g_1 = \frac{\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^3}{\left(\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^2\right)^{3/2}}.
  $$

- **Curtosis poblacional**:  
  $$
  \gamma_2 = \frac{\mathbb{E}\bigl[(X - \mu)^4\bigr]}{\sigma^4}.
  $$
- **Curtosis muestral (no corregida)**:  
  $$
  g_2 = \frac{\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^4}{\left(\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^2\right)^{2}}.
  $$
- **Excess kurtosis (poblacional)**: $$\gamma_2 - 3$$.  
- **Excess kurtosis (muestral)**: $$g_2 - 3$$.

---

### Notas adicionales

- En la práctica, muchas librerías de análisis de datos (por ejemplo, `pandas`, `scipy.stats`, `statsmodels`) implementan funciones para calcular skewness y kurtosis, con opciones de corrección de sesgo para muestras pequeñas.
- Para muestras pequeñas ($$n$$ bajo), conviene revisar si el estimador requiere corrección: algunas definiciones introducen factores multiplicativos en función de $$n$$ para que el estimador sea insesgado.
- Interpretación:  
  - Skewness alto (en valor absoluto) sugiere distribución asimétrica, lo que puede afectar la media y la mediana.  
  - Curtosis alta indica riesgo de valores extremos; curtosis baja sugiere cola más finita.  
- En análisis exploratorio, complementa con histogramas, gráficos de densidad y boxplots para visualizar asimetría y colas.


In [ ]:
## Cálculo de estadísticas de luz: media, desviación, skewness, kurtosis;
## Procesamiento por lotes de datos de curvas de luz; Operaciones matemáticas con numpy, cálculos estadísticos;
## Carga de datos con pandas
## ----------------------------------------------------------------
# Extracción de estadísticas incrementales
def init_stats():
    return {'n': 0, 'mean': 0.0, 'M2': 0.0, 'M3': 0.0, 'M4': 0.0, 'min': np.inf, 'max': -np.inf, 'det_sum': 0}

# Calculos de momentos
def update_moments(s, x, detected):
    n1 = s['n']
    n = n1 + 1
    delta = x - s['mean']
    delta_n = delta / n
    mean = s['mean'] + delta_n
    M2 = s['M2'] + delta * (x - mean)
    M3 = s['M3'] + delta * (x - mean) * (n1 - 1) - 3 * delta_n * s['M2'] if n1 > 0 else 0.0
    M4 = s['M4'] + delta * (x - mean) * (delta * (x - mean) * (n1**2 - 3*n1 + 3) / n**2 + 6 * s['M2'] / n) if n1 > 0 else 0.0
    s['n'] = n; s['mean'] = mean; s['M2'] = M2; s['M3'] = M3; s['M4'] = M4
    s['min'] = min(s['min'], x); s['max'] = max(s['max'], x); s['det_sum'] += detected

stats = {}

# Creación y modificación de dataframe de entrenamiento
for chunk in pd.read_csv('training_set.csv.zip', compression='zip', chunksize=10**6):
    for idx, row in chunk.iterrows():
        obj = int(row['object_id'])
        mjd = row['mjd']
        band = int(row['passband'])
        flux = row['flux']
        detected = int(row['detected'])
        if obj not in stats:
            stats[obj] = {'t_min': mjd, 't_max': mjd, 'bands': set(), 'global': init_stats(), 'band_stats': {}}
        obj_stats = stats[obj]
        obj_stats['t_min'] = min(obj_stats['t_min'], mjd); obj_stats['t_max'] = max(obj_stats['t_max'], mjd)
        update_moments(obj_stats['global'], flux, detected)
        if band not in obj_stats['band_stats']:
            obj_stats['band_stats'][band] = init_stats()
        update_moments(obj_stats['band_stats'][band], flux, detected)
        obj_stats['bands'].add(band)

feature_list = []
meta = pd.read_csv('training_set_metadata.csv')

# Ciclo de calculo y guardado de datos
for idx, row in meta.iterrows():
    obj = int(row['object_id'])
    f = {'object_id': obj}
    if obj in stats:
        obj_stats = stats[obj]
        n = obj_stats['global']['n']; mean = obj_stats['global']['mean']; M2 = obj_stats['global']['M2']
        M3 = obj_stats['global']['M3']; M4 = obj_stats['global']['M4']
        if n > 2 and M2 > 0:
            skew = (np.sqrt(n) * M3) / (M2 ** 1.5)
            kurtosis = (n * M4) / (M2 * M2) - 3
        else:
            skew = 0.0; kurtosis = 0.0
        f['n_obs'] = n; f['t_span'] = obj_stats['t_max'] - obj_stats['t_min']
        f['flux_all_mean'] = mean; f['flux_all_std'] = np.sqrt(M2/(n-1)) if n>1 else 0.0
        f['flux_all_skew'] = skew; f['flux_all_kurtosis'] = kurtosis
        f['flux_det_frac'] = obj_stats['global']['det_sum']/n if n>0 else 0.0
        f['n_bands'] = len(obj_stats['bands'])
        for band in range(6):
            prefix = f"pb{band}"
            if band in obj_stats['band_stats']:
                bs = obj_stats['band_stats'][band]
                nb = bs['n']; mean_b = bs['mean']; M2_b = bs['M2']
                M3_b = bs['M3']; M4_b = bs['M4']
                if nb > 2 and M2_b > 0:
                    skew_b = (np.sqrt(nb) * M3_b) / (M2_b ** 1.5)
                    kurt_b = (nb * M4_b) / (M2_b * M2_b) - 3
                else:
                    skew_b = 0.0; kurt_b = 0.0
                f[f'{prefix}_mean'] = mean_b
                f[f'{prefix}_std'] = np.sqrt(M2_b/(nb-1)) if nb>1 else 0.0
                f[f'{prefix}_min'] = bs['min']; f[f'{prefix}_max'] = bs['max']
                f[f'{prefix}_skew'] = skew_b; f[f'{prefix}_kurtosis'] = kurt_b
                f[f'{prefix}_det_frac'] = bs['det_sum']/nb if nb>0 else 0.0
            else:
                f[f'{prefix}_mean'] = 0.0; f[f'{prefix}_std']=0.0; f[f'{prefix}_min']=0.0
                f[f'{prefix}_max']=0.0; f[f'{prefix}_skew']=0.0; f[f'{prefix}_kurtosis']=0.0; f[f'{prefix}_det_frac']=0.0
    else:
        f.update({'n_obs':0,'t_span':0,'flux_all_mean':0.0,'flux_all_std':0.0,'flux_all_skew':0.0,
                  'flux_all_kurtosis':0.0,'flux_det_frac':0.0,'n_bands':0})
        for band in range(6):
            prefix = f"pb{band}"
            f[f'{prefix}_mean'] = 0.0; f[f'{prefix}_std']=0.0; f[f'{prefix}_min']=0.0
            f[f'{prefix}_max']=0.0; f[f'{prefix}_skew']=0.0; f[f'{prefix}_kurtosis']=0.0; f[f'{prefix}_det_frac']=0.0
    feature_list.append(f)

features_df = pd.DataFrame(feature_list)
data = meta.merge(features_df, on='object_id', how='left')
for col in data.columns:
    if data[col].dtype in [np.float64, np.int64] and data[col].isnull().any():
        data[col].fillna(data[col].median(), inplace=True)

In [ ]:
## Entrenamiento de modelo MLPClassifier; ;
## Transformación de datos;
## División de datos en conjuntos de entrenamiento y validación; Evaluación del modelo con métricas de precisión y reporte;
## Generación de reporte de clasificación; Codificación de etiquetas de clase para target
## ----------------------------------------------------------------
X = data.drop(columns=['object_id','ra','decl','gal_l','gal_b','target'])
y = data['target']
le = LabelEncoder()
y_enc = le.fit_transform(y)
X_train, X_val, y_train, y_val = train_test_split(X, y_enc, test_size=0.2, stratify=y_enc, random_state=42)
## Escalado de características
## Ajuste y transformación de escalador para datos de entrenamiento
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

mlp = MLPClassifier(hidden_layer_sizes=(128,64,32), activation='relu', solver='adam', batch_size=32, max_iter=50, random_state=42, verbose=True)
mlp.fit(X_train_scaled, y_train)
y_val_pred = mlp.predict(X_val_scaled)
print("Accuracy en validación:", accuracy_score(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred, target_names=[str(c) for c in le.classes_]))

joblib.dump(mlp, 'mlp_plasticc_model.pkl')
joblib.dump(le, 'label_encoder.pkl')
joblib.dump(scaler, 'scaler.pkl')

# Con datos extra

En esta sección se quiere incluir los damas datos para comparar su importancia en los resultados obtenidos.

In [4]:
## Cálculo de estadísticas de luz: media, desviación, skewness, kurtosis;
## Procesamiento por lotes de datos de curvas de luz; Operaciones matemáticas con numpy, cálculos estadísticos; Carga de datos con pandas
## ----------------------------------------------------------------
# 1. Carga de metadata
meta = pd.read_csv('training_set_metadata.csv')

# 2. Leer light-curve y extraer estadísticas incrementales con limpieza
def init_stats():
    return {'n': 0, 'mean': 0.0, 'M2': 0.0, 'M3': 0.0, 'M4': 0.0, 'min': np.inf, 'max': -np.inf, 'det_sum': 0}

def update_moments(s, x):
    # Asume aquí que x es valor de flujo ya filtrado; no incluye 'detected' porque
    # sólo llamamos si detected==1.
    n1 = s['n']
    n = n1 + 1
    delta = x - s['mean']
    delta_n = delta / n
    mean = s['mean'] + delta_n
    M2 = s['M2'] + delta * (x - mean)
    M3 = s['M3'] + delta * (x - mean) * (n1 - 1) - 3 * delta_n * s['M2'] if n1 > 0 else 0.0
    M4 = s['M4'] + delta * (x - mean) * (delta * (x - mean) * (n1**2 - 3*n1 + 3) / n**2 + 6 * s['M2'] / n) if n1 > 0 else 0.0
    s['n'] = n; s['mean'] = mean; s['M2'] = M2; s['M3'] = M3; s['M4'] = M4
    s['min'] = min(s['min'], x); s['max'] = max(s['max'], x)
    # s['det_sum'] se incrementa externamente al llamar sólo si detected==1

stats = {}
# Primera pasada: extraer stats por objeto, filtrando no-detecciones
for chunk in pd.read_csv('training_set.csv.zip', compression='zip', chunksize=10**6):
    # Si existe flux_err y queremos filtrar, podríamos:
    # threshold_flux_err = some_value  # calcular de antemano o heurístico
    for idx, row in chunk.iterrows():
        obj = int(row['object_id'])
        mjd = row['mjd']
        band = int(row['passband'])
        flux = row['flux']
        detected = int(row['detected'])
        flux_err = row.get('flux_err', None)  # si existe columna
        # 1) Excluir no-detecciones
        if detected != 1:
            continue
        # 2) Opcional: filtrar por flux_err alto o flux negativo extremo
        # if flux_err is not None and flux_err > threshold_flux_err:
        #     continue
        # if flux < LOWER or flux > UPPER: continue
        if obj not in stats:
            stats[obj] = {'t_min': mjd, 't_max': mjd, 'bands': set(),
                          'global': init_stats(), 'band_stats': {}}
        obj_stats = stats[obj]
        obj_stats['t_min'] = min(obj_stats['t_min'], mjd)
        obj_stats['t_max'] = max(obj_stats['t_max'], mjd)
        # actualizar estadísticos globales
        update_moments(obj_stats['global'], flux)
        obj_stats['global']['det_sum'] = obj_stats['global'].get('det_sum', 0) + 1
        # band_stats
        if band not in obj_stats['band_stats']:
            obj_stats['band_stats'][band] = init_stats()
        update_moments(obj_stats['band_stats'][band], flux)
        obj_stats['band_stats'][band]['det_sum'] = obj_stats['band_stats'][band].get('det_sum', 0) + 1
        obj_stats['bands'].add(band)

# 3. Construcción de DataFrame de features
feature_list = []
for idx, row in meta.iterrows():
    obj = int(row['object_id'])
    f = {'object_id': obj}
    if obj in stats:
        obj_stats = stats[obj]
        n = obj_stats['global']['n']
        M2 = obj_stats['global']['M2']
        M3 = obj_stats['global']['M3']
        M4 = obj_stats['global']['M4']
        # Calcular skew y kurtosis robustos si es posible
        if n > 2 and M2 > 0:
            skew = (np.sqrt(n) * M3) / (M2 ** 1.5)
            kurtosis = (n * M4) / (M2 * M2) - 3
        else:
            skew = 0.0; kurtosis = 0.0
        f['n_obs'] = n
        f['t_span'] = obj_stats['t_max'] - obj_stats['t_min']
        f['flux_all_mean'] = obj_stats['global']['mean']
        f['flux_all_std'] = np.sqrt(M2/(n-1)) if n>1 else 0.0
        f['flux_all_skew'] = skew
        f['flux_all_kurtosis'] = kurtosis
        # fracción de detecciones: en PLAsTiCC, si n es conteo de detecciones tras filtrar,
        # flux_det_frac podría equivaler a 1.0; si se quiere relación con total de observaciones,
        # habría que llevar otro contador.
        f['flux_det_frac'] = obj_stats['global']['det_sum'] / n if n>0 else 0.0
        f['n_bands'] = len(obj_stats['bands'])
        for band in range(6):
            prefix = f"pb{band}"
            if band in obj_stats['band_stats']:
                bs = obj_stats['band_stats'][band]
                nb = bs['n']
                M2_b = bs['M2']
                M3_b = bs['M3']
                M4_b = bs['M4']
                if nb > 2 and M2_b > 0:
                    skew_b = (np.sqrt(nb) * M3_b) / (M2_b ** 1.5)
                    kurt_b = (nb * M4_b) / (M2_b * M2_b) - 3
                else:
                    skew_b = 0.0; kurt_b = 0.0
                f[f'{prefix}_mean'] = bs['mean']
                f[f'{prefix}_std'] = np.sqrt(M2_b/(nb-1)) if nb>1 else 0.0
                f[f'{prefix}_min'] = bs['min']
                f[f'{prefix}_max'] = bs['max']
                f[f'{prefix}_skew'] = skew_b
                f[f'{prefix}_kurtosis'] = kurt_b
                f[f'{prefix}_det_frac'] = bs['det_sum'] / nb if nb>0 else 0.0
            else:
                f[f'{prefix}_mean'] = 0.0; f[f'{prefix}_std']=0.0; f[f'{prefix}_min']=0.0
                f[f'{prefix}_max']=0.0; f[f'{prefix}_skew']=0.0; f[f'{prefix}_kurtosis']=0.0; f[f'{prefix}_det_frac']=0.0
    else:
        # Sin datos de serie temporal tras filtrar; rellenar con ceros/neutros
        f.update({'n_obs':0,'t_span':0,'flux_all_mean':0.0,'flux_all_std':0.0,
                  'flux_all_skew':0.0,'flux_all_kurtosis':0.0,'flux_det_frac':0.0,'n_bands':0})
        for band in range(6):
            prefix = f"pb{band}"
            f[f'{prefix}_mean'] = 0.0; f[f'{prefix}_std']=0.0; f[f'{prefix}_min']=0.0
            f[f'{prefix}_max']=0.0; f[f'{prefix}_skew']=0.0; f[f'{prefix}_kurtosis']=0.0; f[f'{prefix}_det_frac']=0.0
    feature_list.append(f)

features_df = pd.DataFrame(feature_list)

# 4. Merge con metadata
data = meta.merge(features_df, on='object_id', how='left')

# 5. Incorporar las nuevas características de redshift/extinción:
# Asumiendo que meta contiene columnas: hostgal_specz, hostgal_photoz, hostgal_photoz_err, distmod, mwebv
# a) Crear columna redshift_used apuntando a specz cuando exista, sino a photoz:
data['redshift_used'] = data['hostgal_specz'].fillna(data['hostgal_photoz'])
# Si hay NaN en redshift_used, rellenar con mediana:
data['redshift_used'].fillna(data['redshift_used'].median(), inplace=True)
# b) Rellenar NaNs en hostgal_photoz, hostgal_photoz_err, distmod, mwebv con medianas:
for col in ['hostgal_photoz', 'hostgal_photoz_err', 'distmod', 'mwebv']:
    if col in data.columns:
        data[col].fillna(data[col].median(), inplace=True)



<ipython-input-4-2413030156>:132: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['redshift_used'].fillna(data['redshift_used'].median(), inplace=True)
<ipython-input-4-2413030156>:136: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, 

In [5]:
## Entrenamiento de modelo MLPClassifier; Escalado de características;
## Ajuste y transformación de escalador para datos de entrenamiento;
## Transformación de datos; División de datos en conjuntos de entrenamiento y validación; Evaluación del modelo con métricas de precisión y reporte; Generación de reporte de clasificación; Codificación de etiquetas de clase para target
## ----------------------------------------------------------------
# 6. Preparar X e y
# Eliminar columnas no deseadas. Ahora NO eliminar hostgal_*, distmod, mwebv ni redshift_used.
drop_cols = ['object_id', 'ra', 'decl', 'gal_l', 'gal_b', 'target']
X = data.drop(columns=[c for c in drop_cols if c in data.columns])
y = data['target']

# 7. Encoding y y división train/val
le = LabelEncoder()
y_enc = le.fit_transform(y)
X_train, X_val, y_train, y_val = train_test_split(X, y_enc, test_size=0.2,
                                                  stratify=y_enc, random_state=42)

# 8. Escalado
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 9. Entrenamiento del modelo (ejemplo MLP; se puede experimentar con RF, etc.)
mlp = MLPClassifier(hidden_layer_sizes=(128,64,32), activation='relu',
                    solver='adam', batch_size=32, max_iter=50,
                    random_state=42, verbose=True)
mlp.fit(X_train_scaled, y_train)
y_val_pred = mlp.predict(X_val_scaled)
print("Accuracy en validación:", accuracy_score(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred, target_names=[str(c) for c in le.classes_]))

# 10. Guardar modelos
joblib.dump(mlp, 'mlp_plasticc_model.pkl')
joblib.dump(le, 'label_encoder.pkl')
joblib.dump(scaler, 'scaler.pkl')

Iteration 1, loss = 1.48098563
Iteration 2, loss = 0.95101788
Iteration 3, loss = 0.84764953
Iteration 4, loss = 0.77621076
Iteration 5, loss = 0.73693989
Iteration 6, loss = 0.70288342
Iteration 7, loss = 0.68609865
Iteration 8, loss = 0.64840002
Iteration 9, loss = 0.62588463
Iteration 10, loss = 0.60997748
Iteration 11, loss = 0.59291472
Iteration 12, loss = 0.59360703
Iteration 13, loss = 0.58395067
Iteration 14, loss = 0.56981134
Iteration 15, loss = 0.54844911
Iteration 16, loss = 0.52528831
Iteration 17, loss = 0.51310189
Iteration 18, loss = 0.52815115
Iteration 19, loss = 0.49485277
Iteration 20, loss = 0.48535803
Iteration 21, loss = 0.47554739
Iteration 22, loss = 0.46617277
Iteration 23, loss = 0.45487068
Iteration 24, loss = 0.45044855
Iteration 25, loss = 0.45319525
Iteration 26, loss = 0.43375569
Iteration 27, loss = 0.42364822
Iteration 28, loss = 0.42059828
Iteration 29, loss = 0.41578635
Iteration 30, loss = 0.40788168
Iteration 31, loss = 0.41916985
Iteration 32, los

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


['scaler.pkl']

Graficos de los objetos en cada filtro de cada Target

In [20]:
## Operaciones: valores_unicos = meta['target'].unique()
## ----------------------------------------------------------------
# Graficos


import zipfile
import pandas as pd
import matplotlib.pyplot as plt

# Definir función para graficar lightcurve de un ID y passband específicos
def plot_lightcurve_for_id_passband(df, id_value, passband_value, target):
    subset_id = df[df['object_id'] == id_value]
    subset = subset_id[subset_id['passband'] == passband_value]

    if subset.empty:
        print(f"No se encontraron datos para object_id = {id_value} y passband = {passband_value}.")
        return

    plt.figure()
    plt.errorbar(subset['mjd'], subset['flux'], yerr=subset['flux_err'], fmt='o', elinewidth=1, capsize=2)
    plt.xlabel('MJD')
    plt.ylabel('Flux')
    plt.title(f'Lightcurve: object_id={id_value}, Band={passband_value}, Target= {target}')
    plt.grid(True)

    plt.savefig(
    f"/content/figuras/objeto{id_value}_banda{passband_value}.jpg",
    transparent=False,
    bbox_inches='tight',
    facecolor='white'
    )
    plt.close()

lc = pd.read_csv('training_set.csv.zip', compression='zip')
meta = pd.read_csv('training_set_metadata.csv')

valores_unicos = meta['target'].unique()
idselect = np.zeros(len(valores_unicos))

# Lo uso para verificar como es la curva de luz de los target
for i in range(len(valores_unicos)):
    idselect[i] = meta[meta["target"] == valores_unicos[i]]["object_id"].iloc[0]
    dtu = lc[lc["object_id"] == idselect[i]]
    ban = dtu["passband"].unique()
    for j in range(len(ban)):
      plot_lightcurve_for_id_passband(dtu, idselect[i], ban[j], valores_unicos[i])

# Testeo de datos

In [17]:
## Escalado de características; Cálculo de estadísticas de luz: media, desviación, skewness, kurtosis;
## Procesamiento por lotes de datos de curvas de luz; Codificación de etiquetas de clase para target;
## Operaciones matemáticas con numpy, cálculos estadísticos; Carga de datos con pandas
## ----------------------------------------------------------------


# 1. Cargar el modelo, el label encoder y el scaler que guardaste
mlp = joblib.load('mlp_plasticc_model.pkl')           # tu modelo entrenado
le = joblib.load('label_encoder.pkl')                 # LabelEncoder ajustado en train
scaler = joblib.load('scaler.pkl')                    # StandardScaler ajustado en train

# 2. Leer metadata de test
# Ajusta la ruta si es distinto:
metadata_test = pd.read_csv('test_set_metadata.csv.zip', compression='zip')
metadata_test = metadata_test.head(1000)
ids_validos = set(metadata_test['object_id'].values)
# Esperamos que metadata_test tenga columna 'object_id' y posibles columnas extra (por si usas hostgal_*, etc.)
# 3. Extraer estadísticas de curvas de luz de test
# Definimos funciones de estadísticos incrementales (idénticas a las de tu entrenamiento)
def init_stats():
    return {'n': 0, 'mean': 0.0, 'M2': 0.0, 'M3': 0.0, 'M4': 0.0, 'min': np.inf, 'max': -np.inf, 'det_sum': 0}

def update_moments(s, x, detected=None):
    # Si en tu entrenamiento usaste detected para global['det_sum'], aquí también:
    n1 = s['n']
    n = n1 + 1
    delta = x - s['mean']
    delta_n = delta / n
    mean = s['mean'] + delta_n
    M2 = s['M2'] + delta * (x - mean)
    if n1 > 0:
        M3 = s['M3'] + delta * (x - mean) * (n1 - 1) - 3 * delta_n * s['M2']
        M4 = s['M4'] + delta * (x - mean) * (delta * (x - mean) * (n1**2 - 3*n1 + 3) / n**2 + 6 * s['M2'] / n)
    else:
        M3 = 0.0
        M4 = 0.0
    s['n'] = n
    s['mean'] = mean
    s['M2'] = M2
    s['M3'] = M3
    s['M4'] = M4
    s['min'] = min(s['min'], x)
    s['max'] = max(s['max'], x)
    # actualizar suma de detecciones si proporcionaste detected
    if detected is not None:
        s['det_sum'] = s.get('det_sum', 0) + int(detected)

# Recorremos todos los archivos de test de curvas de luz. Por ejemplo, si hay test_set_batch1.csv.zip, test_set_batch2...
# Ajusta la lista según tus archivos:
test_curve_files = ['test_set_batch1.csv.zip']  # agrega más si es necesario

# stats_test: dict con key=object_id, value: dict con t_min, t_max, bands, global stats, band_stats
stats_test = {}
for file in test_curve_files:
    # Leer en chunks (por ejemplo, chunksize=10**6 si es grande)
    for chunk in pd.read_csv(file, compression='zip', chunksize=10**6):
        # Asegúrate de que las columnas coincidan: object_id, mjd, passband, flux, detected
        chunk = chunk[chunk['object_id'].isin(ids_validos)]
        for idx, row in chunk.iterrows():
            obj = int(row['object_id'])
            mjd = row['mjd']
            band = int(row['passband'])
            flux = row['flux']
            detected = row.get('detected', None)  # si existe; en test a veces no hay etiqueta, pero normalmente sí hay columna detected
            # Si en tu pipeline de entrenamiento no filtrabas por detected (tomabas todos), seguimos igual.
            if obj not in stats_test:
                stats_test[obj] = {
                    't_min': mjd,
                    't_max': mjd,
                    'bands': set(),
                    'global': init_stats(),
                    'band_stats': {}
                }
            obj_stats = stats_test[obj]
            obj_stats['t_min'] = min(obj_stats['t_min'], mjd)
            obj_stats['t_max'] = max(obj_stats['t_max'], mjd)
            # actualizar global
            update_moments(obj_stats['global'], flux, detected)
            # band stats
            if band not in obj_stats['band_stats']:
                obj_stats['band_stats'][band] = init_stats()
            update_moments(obj_stats['band_stats'][band], flux, detected)
            obj_stats['bands'].add(band)

# 4. Construir DataFrame de características para test
feature_list = []
for idx, row in metadata_test.iterrows():
    obj = int(row['object_id'])
    f = {'object_id': obj}
    if obj in stats_test:
        obj_stats = stats_test[obj]
        # Estadísticos globales
        n = obj_stats['global']['n']
        M2 = obj_stats['global']['M2']
        M3 = obj_stats['global']['M3']
        M4 = obj_stats['global']['M4']
        # Cálculo de skew y kurtosis análogo al entrenamiento
        if n > 2 and M2 > 0:
            skew = (np.sqrt(n) * M3) / (M2 ** 1.5)
            kurtosis = (n * M4) / (M2 * M2) - 3
        else:
            skew = 0.0
            kurtosis = 0.0
        f['n_obs'] = n
        f['t_span'] = obj_stats['t_max'] - obj_stats['t_min']
        f['flux_all_mean'] = obj_stats['global']['mean']
        f['flux_all_std'] = np.sqrt(M2/(n-1)) if n > 1 else 0.0
        f['flux_all_skew'] = skew
        f['flux_all_kurtosis'] = kurtosis
        # fracción de detecciones
        det_sum = obj_stats['global'].get('det_sum', None)
        if det_sum is not None and n > 0:
            f['flux_det_frac'] = det_sum / n
        else:
            # Si en test no viene 'detected' o no la usaste, podrías fijar 1.0 o 0.0 según convenga. Aquí:
            f['flux_det_frac'] = obj_stats['global'].get('det_sum', 0) / n if n > 0 else 0.0
        f['n_bands'] = len(obj_stats['bands'])
        # Estadísticos por banda (suponiendo 6 bandas, 0..5)
        for band in range(6):
            prefix = f"pb{band}"
            if band in obj_stats['band_stats']:
                bs = obj_stats['band_stats'][band]
                nb = bs['n']
                M2_b = bs['M2']
                M3_b = bs['M3']
                M4_b = bs['M4']
                if nb > 2 and M2_b > 0:
                    skew_b = (np.sqrt(nb) * M3_b) / (M2_b ** 1.5)
                    kurt_b = (nb * M4_b) / (M2_b * M2_b) - 3
                else:
                    skew_b = 0.0
                    kurt_b = 0.0
                f[f'{prefix}_mean'] = bs['mean']
                f[f'{prefix}_std'] = np.sqrt(M2_b/(nb-1)) if nb > 1 else 0.0
                f[f'{prefix}_min'] = bs['min']
                f[f'{prefix}_max'] = bs['max']
                f[f'{prefix}_skew'] = skew_b
                f[f'{prefix}_kurtosis'] = kurt_b
                det_sum_b = bs.get('det_sum', None)
                if det_sum_b is not None and nb > 0:
                    f[f'{prefix}_det_frac'] = det_sum_b / nb
                else:
                    f[f'{prefix}_det_frac'] = bs.get('det_sum', 0) / nb if nb > 0 else 0.0
            else:
                # Sin observaciones en esta banda
                f[f'{prefix}_mean'] = 0.0
                f[f'{prefix}_std'] = 0.0
                f[f'{prefix}_min'] = 0.0
                f[f'{prefix}_max'] = 0.0
                f[f'{prefix}_skew'] = 0.0
                f[f'{prefix}_kurtosis'] = 0.0
                f[f'{prefix}_det_frac'] = 0.0
    else:
        # Objeto sin curvas en test (raro, pero para robustez)
        f.update({
            'n_obs': 0,
            't_span': 0,
            'flux_all_mean': 0.0,
            'flux_all_std': 0.0,
            'flux_all_skew': 0.0,
            'flux_all_kurtosis': 0.0,
            'flux_det_frac': 0.0,
            'n_bands': 0
        })
        for band in range(6):
            prefix = f"pb{band}"
            f[f'{prefix}_mean'] = 0.0
            f[f'{prefix}_std'] = 0.0
            f[f'{prefix}_min'] = 0.0
            f[f'{prefix}_max'] = 0.0
            f[f'{prefix}_skew'] = 0.0
            f[f'{prefix}_kurtosis'] = 0.0
            f[f'{prefix}_det_frac'] = 0.0
    feature_list.append(f)

features_test_df = pd.DataFrame(feature_list)

# 5. Merge con metadata_test
data_test = metadata_test.merge(features_test_df, on='object_id', how='left')

# 6. Incorporar columnas extra si usas hostgal_specz, hostgal_photoz, distmod, mwebv, etc.
# Si en test se incluyen, repite mismo procesamiento que en entrenamiento:
if 'hostgal_specz' in data_test.columns and 'hostgal_photoz' in data_test.columns:
    data_test['redshift_used'] = data_test['hostgal_specz'].fillna(data_test['hostgal_photoz'])
    data_test['redshift_used'].fillna(data_test['redshift_used'].median(), inplace=True)
    for col in ['hostgal_photoz', 'hostgal_photoz_err', 'distmod', 'mwebv']:
        if col in data_test.columns:
            data_test[col].fillna(data_test[col].median(), inplace=True)

# 7. Seleccionar X_test con mismas columnas que en entrenamiento
# Recuerda: en entrenamiento habías hecho:
# X = data.drop(columns=['object_id','ra','decl','gal_l','gal_b','target'])
# Aquí, en test no habrá columna 'target'. Asegúrate de dropear las mismas columnas redundantes.
cols_drop = ['object_id', 'ra', 'decl', 'gal_l', 'gal_b', 'target']
X_test = data_test.drop(columns=[c for c in cols_drop if c in data_test.columns])

# 8. Imputar NaNs en X_test: rellenar con medianas de cada columna (calculadas en entrenamiento idealmente).
# OJO: si quieres máxima fidelidad, deberías usar en cada columna la misma mediana que usaste en entrenamiento.
# Pero si no almacenaste esas medianas, una aproximación es rellenar con la mediana de X_test.
for col in X_test.columns:
    if X_test[col].dtype in [np.float64, np.int64] and X_test[col].isnull().any():
        med = X_test[col].median()
        X_test[col].fillna(med, inplace=True)

# 9. Escalado usando el StandardScaler entrenado:
X_test_scaled = scaler.transform(X_test)

# 10. predecir probabilidades
proba = mlp.predict_proba(X_test_scaled)  # array shape (n_objetos_test, 14)

# 11. Construir DataFrame de salida
# Obtener nombres de clases desde LabelEncoder. Por ejemplo, le.classes_ podría ser array(['15', '42', ...]) o int/string.
class_names = [f"Class_{cls}" for cls in le.classes_]  # conviertele a str para usar en columnas
# O, si prefieres nombres como "class_0","class_1",...:
# class_names = [f'class_{i}' for i in range(len(le.classes_))]

df_out = pd.DataFrame(proba, columns=class_names)
df_out.insert(0, 'object_id', data_test['object_id'].values)
df_out.insert(1, 'class_99', 1 - df_out[class_names].sum(axis=1))
rdf_out = df_out.round(3)

# 12. (Opcional) Si la competición pide otra forma (por ejemplo, en formato long: filas duplicadas para cada target),
# puedes transformar df_out en long con pd.melt, pero la mayoría de competiciones PLAsTiCC esperan wide:
# object_id, prob_class0, prob_class1, ..., prob_class13.

# 13. Guardar el CSV de Submission
rdf_out.to_csv('submission.csv', index=False)

# 14. Mostrar un vistazo
rdf_out


<ipython-input-17-4170308700>:185: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_test['redshift_used'].fillna(data_test['redshift_used'].median(), inplace=True)
<ipython-input-17-4170308700>:188: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].me

,object_id,class_99,Class_6,Class_15,Class_16,Class_42,Class_52,Class_53,Class_62,Class_64,Class_65,Class_67,Class_88,Class_90,Class_92,Class_95
0,13,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.234,0.0,0.766,0.0,0.0,0.0
1,14,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,1.000,0.0,0.0,0.0
2,17,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,1.000,0.0,0.0,0.0
3,23,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.292,0.0,0.708,0.0,0.0,0.0
4,34,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1.000,0.0,0.000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10180,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,1.000,0.0,0.0,0.0
996,10213,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,1.000,0.0,0.0,0.0
997,10218,0.0,0.0,0.389,0.0,0.0,0.0,0.0,0.0,0.0,0.005,0.0,0.606,0.0,0.0,0.0
998,10228,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.062,0.0,0.938,0.0,0.0,0.0


In [ ]:
## Operaciones: df_out.to_csv('sample_submission_check.csv', index
## ----------------------------------------------------------------
df_out.to_csv('sample_submission_check.csv', index=False)

#### Este fue un intento fallido de analizar propiedades estadisticas por seciones de la curva de Luz

In [ ]:
## Carga de datos con pandas
## ----------------------------------------------------------------
lc = pd.read_csv('training_set.csv.zip', compression='zip')
meta = pd.read_csv('training_set_metadata.csv')

id_vu = meta["object_id"].unique()

nombres  = ["mean", "std", "skew", "kur"]
arrC = [ f"{k}{nombres[i]}band{j}"  for j in range(6) for i in range(len(nombres))  for k in range(3) ]
dfstat = pd.DataFrame(columns=arrC)

listfil = []

for i in range(len(id_vu)):
  aux = []
  dtu = lc[lc["object_id"] == id_vu[i]]
  ban = np.sort( dtu["passband"].unique())
  for j in range(len(ban)):
    flux = np.array(dtu[dtu["passband"] == ban[j]]["flux"].tolist())
    mjd = np.array(dtu[dtu["passband"] == ban[j]]["mjd"].tolist())
    res = np.zeros(len(mjd) - 1)

    for k in range(len(mjd) - 1):
      res[k] = np.abs(mjd[k] - mjd[k+1])

    if len(res) < 6:
      break

    a = np.sort(res)[-2:len(res)]
    f = [list(res).index(a[0]), list(res).index(a[1])]

    flux1 = flux [mjd < mjd[f[1]]]

    flux2 = flux[ [mjd > mjd[f[1]]][0] * [mjd < mjd[f[0]]][0] ]

    flux3 = flux [ mjd > mjd[f[0]] ]

    mean = [ np.mean(flux1),np.mean(flux2), np.mean(flux3) ]
    des = [ np.std(flux1), np.std(flux2), np.std(flux3) ]
    kt = [ kurtosis(flux1), kurtosis(flux2), kurtosis(flux3) ]
    sk = [ skew(flux1), skew(flux2), skew(flux3)]

    stats = mean+des+kt+sk
    stats_sin_nan = [0 if np.isnan(x) else x for x in stats]

    aux += stats_sin_nan

  if len(aux) == 72:
    dfstat = pd.concat([dfstat, pd.DataFrame([aux], columns=arrC)], ignore_index=True)

  else:
    listfil.append(id_vu[i])

##Filtro de datos pos procesado

metaf = meta[~meta["object_id"].isin(listfil)]
com = pd.concat([metaf, dfstat], axis = 1)
rdfstat = dfstat.reset_index(drop=True)
rmetaf = metaf.reset_index(drop=True)
combine = pd.concat([rmetaf, rdfstat], axis = 1)
combine_sin_nan = combine[~combine.isna().any(axis=1)]

In [ ]:
## Entrenamiento de modelo MLPClassifier; Escalado de características; Ajuste y transformación de escalador para datos de entrenamiento; Transformación de datos; División de datos en conjuntos de entrenamiento y validación; Evaluación del modelo con métricas de precisión y reporte; Generación de reporte de clasificación; Codificación de etiquetas de clase para target
## ----------------------------------------------------------------
# parte de ML

drop_cols = ['object_id', 'ra', 'decl', 'gal_l', 'gal_b', 'target']
X = combine_sin_nan.drop(columns=[c for c in drop_cols if c in combine_sin_nan.columns])
y = combine_sin_nan['target']


# 7. Encoding y y división train/val
le = LabelEncoder()
y_enc = le.fit_transform(y)
X_train, X_val, y_train, y_val = train_test_split(X, y_enc, test_size=0.2,
                                                  stratify=y_enc, random_state=42)

# 8. Escalado
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 9. Entrenamiento del modelo (ejemplo MLP; se puede experimentar con RF, etc.)
mlp = MLPClassifier(hidden_layer_sizes=(256,128,64,32), activation='relu',
                    solver='adam', batch_size=32, max_iter=50,
                    random_state=42, verbose=True)
mlp.fit(X_train_scaled, y_train)
y_val_pred = mlp.predict(X_val_scaled)
print("Accuracy en validación:", accuracy_score(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred, target_names=[str(c) for c in le.classes_]))
